In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from keras.models import load_model, Model
from sklearn.metrics import precision_recall_fscore_support

#### **Feature extraction and fusion**

Load pre-trained base models. 

Add a hook to the base model to extract features from fully connnected layers.

In [ ]:
#Load pre-trained base models
mobilenet_model = load_model("saved_model/finetuned_mobilenet.keras")
incepresnet_model = load_model("saved_model/finetuned_incepresnet.keras")
xception_model = load_model("saved_model/finetuned_xception.keras")

In [ ]:
#Add hook to the base models to extracted features from the layer that before the softmax output layer
mobilenet_model = Model(inputs=mobilenet_model.input, outputs=mobilenet_model.get_layer(index=-2).output)
incepresnet_model = Model(inputs=incepresnet_model.input, outputs=incepresnet_model.get_layer(index=-2).output)
xception_model = Model(inputs=xception_model.input, outputs=xception_model.get_layer(index=-2).output)

#Extract training data features using base models
mobilenet_train_prediction = np.array(mobilenet_model.predict(x_train))
mobilenet_Xtrain = np.reshape(mobilenet_train_prediction, (mobilenet_train_prediction.shape[0], mobilenet_train_prediction.shape[1]))

#Extract testing data features using base models
mobilenet_test_prediction = np.array(mobilenet_model.predict(x_test))
mobilenet_Xtest = np.reshape(mobilenet_test_prediction, (mobilenet_test_prediction.shape[0], mobilenet_test_prediction.shape[1]))

#Show the extracted feature size
print('\tMobilenet features training shape: ', mobilenet_Xtrain.shape)
print('\tMobilenet features testing shape: ', mobilenet_Xtest.shape)

incepresnet_train_prediction = np.array(incepresnet_model.predict(x_train))
incepresnet_Xtrain = np.reshape(incepresnet_train_prediction, (incepresnet_train_prediction.shape[0], incepresnet_train_prediction.shape[1]))

incepresnet_test_prediction = np.array(incepresnet_model.predict(x_test))
incepresnet_Xtest = np.reshape(incepresnet_test_prediction, (incepresnet_test_prediction.shape[0], incepresnet_test_prediction.shape[1]))

print('\tIncepresnet features training shape: ', incepresnet_Xtrain.shape)
print('\tIncepresnet features testing shape: ', incepresnet_Xtest.shape)

xception_train_prediction = np.array(xception_model.predict(x_train))
xception_Xtrain = np.reshape(xception_train_prediction, (xception_train_prediction.shape[0], xception_train_prediction.shape[1]))

xception_test_prediction = np.array(xception_model.predict(x_test))
xception_Xtest = np.reshape(xception_test_prediction, (xception_test_prediction.shape[0], xception_test_prediction.shape[1]))

print('\tXception features training shape: ', xception_Xtrain.shape)
print('\tXception features testing shape: ', xception_Xtest.shape)

In [ ]:
#Concatenate training and testing features independently
FEATS = []
Test_FEATS = []

FEATS = np.concatenate((mobilenet_Xtrain, incepresnet_Xtrain, xception_Xtrain), axis=1)
print("- preds shape:", FEATS.shape)

Test_FEATS = np.concatenate((mobilenet_Xtest, incepresnet_Xtest, xception_Xtest), axis=1)
print("- test preds shape:", Test_FEATS.shape)

#### **Feature selection**

Use feature selection methods to filter the concatenated features.

In [ ]:
from sklearn.decomposition import FastICA

# Initialise the ICA model
ica = FastICA(n_components=50) # Change the n_components to the number of features that want to keep
ica.fit(FEATS)

X_train_ica = ica.transform(FEATS)
X_test_ica = ica.transform(Test_FEATS)

X_train_ica.shape

#### **Ensemble learning**

Use processed training features as the input data to train the higher-level ML classifier for ensemble learning, and use processed testing features to test the performance.

In [ ]:
#Initilise with SVM classifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score

print("Classification with Linear SVM ...")
svm_model = LinearSVC(dual="auto", random_state=0, tol=1e-2)
svm_model.fit(X_train_ica, np.ravel(y, order='C'))  # Use the data pre-processing file to acquire corresponding variables
svm_result = svm_model.predict(X_test_ica) # Make prediction with extracted testing features

#Acquire class label for testing
x_pure_test, y_pure_test = create_dataset(test_dataset_dir, categories, img_wid, img_high) # Use the data pre-processing file to acquire corresponding variables

acc = accuracy_score(svm_result, np.ravel(y_pure_test, order='C'))
precision, recall, f1_score, _ = precision_recall_fscore_support(svm_result, y_pure_test, average=None)

#Evaluate the predictions
print("\tAccuracy Linear SVM: %0.4f" % acc)
for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")

In [ ]:
#Initilise with XGBoost classifier
from xgboost import XGBClassifier

# Train XGBoost model
xgb_model = XGBClassifier(booster = 'gbtree', eta = 0.9, max_depth = 10)
xgb_model.fit(X_train_ica, y)  # Use the data pre-processing file to acquire corresponding variable
xgb_result = xgb_model.predict(X_test_ica)  # Make prediction with extracted testing features

#Acquire class label for testing
x_pure_test, y_pure_test = create_dataset(test_dataset_dir, categories, img_wid, img_high) # Use the data pre-processing file to acquire corresponding variables

acc = accuracy_score(xgb_result, np.ravel(y_pure_test, order='C'))
precision, recall, f1_score, _ = precision_recall_fscore_support(xgb_result, y_pure_test, average=None)

#Evaluate the predictions
print("\tAccuracy XGBoost: %0.4f" % acc)
for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")

In [ ]:
#Initilise with RandomForest classifier
from sklearn.ensemble import RandomForestClassifier

# Train RandomForest Classifier
rf_model = RandomForestClassifier(min_samples_split=3, max_features='log2')
rf_model.fit(X_train_ica, y)
rf_result = rf_model.predict(X_test_ica)

#Acquire class label for testing
x_pure_test, y_pure_test = create_dataset(test_dataset_dir, categories, img_wid, img_high)

acc = accuracy_score(rf_result, np.ravel(y_pure_test, order='C'))
precision, recall, f1_score, _ = precision_recall_fscore_support(rf_result, y_pure_test, average=None)

#Evaluate the predictions
print("\tAccuracy RandomForest: %0.4f" % acc)
for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")


In [ ]:
#Initilise with Nive Bayes classifier
from sklearn.naive_bayes import GaussianNB

# Train Naive Bayes Classifier
nb_model = GaussianNB()
nb_model.fit(X_train_ica, y)
nb_result = nb_model.predict(X_test_ica)

#Acquire class label for testing
x_pure_test, y_pure_test = create_dataset(test_dataset_dir, categories, img_wid, img_high)

acc = accuracy_score(nb_result, np.ravel(y_pure_test, order='C'))
precision, recall, f1_score, _ = precision_recall_fscore_support(nb_result, y_pure_test, average=None)

#Evaluate the predictions
print("\tAccuracy NaiveBayes: %0.4f" % acc)
for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")

In [ ]:
#Initilise with k-Nearest Neighbors classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Train k-Nearest Neighbors Classifier
knn_model = KNeighborsClassifier(n_neighbors=3, weights = 'distance', p=1)
knn_model.fit(FEATS, y)
knn_result = knn_model.predict(Test_FEATS)

#Acquire class label for testing
x_pure_test, y_pure_test = create_dataset(test_dataset_dir, categories, img_wid, img_high)

acc = accuracy_score(knn_result, np.ravel(y_pure_test, order='C'))
precision, recall, f1_score, _ = precision_recall_fscore_support(knn_result, y_pure_test, average=None)

#Evaluate the predictions
print("\tAccuracy Linear regression: %0.4f" % acc)
for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")

In [ ]:
#Conduct majority voting with trained ML classifiers
from sklearn.ensemble import VotingClassifier

# Combine predictions into a list
all_predictions = [nb_result, xgb_result, rf_result, knn_result, svm_result]

# Initialise a voting classifier using hard voting (majority voting)
voting_clf = VotingClassifier(estimators=[
    ('naive_bayes', nb_model),
    ('xgboost', xgb_model),
    ('rf', rf_model),
    ('knn', knn_model),
    ('svm', svm_model),
], voting='hard')

# Reshape predictions to ensure they have at most two dimensions
reshaped_predictions = [pred.reshape(-1, 1) for pred in all_predictions]

# Fit the voting classifier on the reshaped predictions
voting_clf.fit(np.hstack(reshaped_predictions), y_pure_test)

# Make predictions using the voting classifier
ensemble_predictions = voting_clf.predict(np.hstack(reshaped_predictions))

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(y_pure_test, ensemble_predictions)
ensemble_precision, ensemble_recall, ensemble_f1_score, _ = precision_recall_fscore_support(y_pure_test, ensemble_predictions, average=None)

print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble recall:", ensemble_recall)
print("Ensemble precision:", ensemble_precision)
print("Ensemble f1:", ensemble_f1_score)

#### **Confusion matrix evaluation**

Evaluate the final predictions with ground truth label, and provide a confusion matrix.

In [ ]:
import matplotlib.pyplot as plt
def cm_plt(ax, cm, classes, cmap, title, normalize, font_size= 16 ):
    im = ax.imshow(cm, interpolation="nearest", cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(
        xticks=np.arange(cm.shape[1]),
        yticks=np.arange(cm.shape[0]),
        # ... and label them with the respective list entries
        xticklabels=classes,
        yticklabels=classes,
        title=title,
        ylabel="True label",
        xlabel="Predicted label",
    )

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor", fontsize=font_size)
    plt.setp(ax.get_yticklabels(), fontsize=font_size)
    # Loop over data dimensions and create text annotations.
    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j,
                i,
                format(cm[i, j], fmt),
                ha="center",
                va="center",
                color="white" if cm[i, j] > thresh else "black",
                fontsize=font_size
            )

    return ax

# Defining function for confusion matrix plot
def plt_confusion_mat(cm, classes, fig_size, cmap=plt.cm.Blues, font_size=16):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=fig_size)
    ax1 = cm_plt(
        ax1,
        cm,
        classes,
        cmap,
        title="Confusion matrix, without normalization",
        normalize=False,
        font_size=font_size
    )

    cmn = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
    ax2 = cm_plt(
        ax2,
        cmn,
        classes,
        cmap,
        title="Normalized confusion matrix",
        normalize=True,
        font_size=font_size
    )

In [ ]:
from sklearn.metrics import confusion_matrix
# Store class label in the sequence of saving data file
Label = ['x','xx'] 
# A sample of Kvasirv2 dataset
Kvasirv2_label = ['Dyed-lifted-polyps', 'Dyed-resection-margins', 'Esophagitis', 'Normal-cecum', 'Normal-pylorus', 'Normal-z-line', 'Polyps', 'Ulcerative-colitis']

# Convert predicted_results and ground_truth_labels to NumPy arrays
predicted_results = np.array(ensemble_predictions)
ground_truth_labels = np.array(y_pure_test)

# Calculate the confusion matrix
cm = confusion_matrix(predicted_results, ground_truth_labels)

# Call the confusion matrix plotting functions
fig_size = (24, 7)  # Adjust the figure size as needed
plt_confusion_mat(cm, Label, fig_size)

#### **t-SNE feature analysis**

Evaluate the whole model's feature classification quality.

In [ ]:
from sklearn.manifold import TSNE

# Perform dimensionality reduction using t-SNE
tsne = TSNE(n_components=2, random_state=42)
embedded_features = tsne.fit_transform(Test_FEATS)

# Visualise the embedded features
plt.figure(figsize=(10, 8))
plt.scatter(embedded_features[:, 0], embedded_features[:, 1], c=y_t, cmap='viridis') # Use the data pre-processing file to acquire corresponding variables
plt.colorbar(label='Classes')
plt.title('t-SNE Visualization of Fused Features')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')

Kvasirv2_label = ['Dyed-lifted-polyps', 'Dyed-resection-margins', 'Esophagitis', 'Normal-cecum', 'Normal-pylorus', 'Normal-z-line', 'Polyps', 'Ulcerative-colitis']

# Add label names as annotations
for i, label in enumerate(Kvasirv2_label):
    plt.annotate(label, (embedded_features[y_t == i, 0].mean(), embedded_features[y_t == i, 1].mean()),  # Use the data pre-processing file to acquire corresponding variables
                 ha='center', va='center', size=10, bbox=dict(facecolor='white', alpha=0.5))